# Make an Antarctic Rift Catalog

## Step 0: Decide which data to use
This is done using IcePyx to spatially window available data

In [1]:
shelf_name = 'amery'

# Format is WSNE
# spatial_extent = [-27.8, -76.1, -3.0, -70.2] # Brunt-Riiser-Ekstrom System
# spatial_extent = [-3.0,-71.5, 39.5, -68.6] # Fimbul
spatial_extent = [67.6, -72.44,74.87,-68.39] # Amery
# spatial_extent = [159, -86, 180, -69] # East Ross
# spatial_extent = [-180, -86, -140, -77] # West Ross
# spatial_extent = [-65.5,-68.7,-60.2,-66] # Larsen C
# spatial_extent = [-82.0, 82.4, -79.5, 82.9] # Milne

In [ ]:
# If filelist_file_name exists, then load it.  Otherwise query icepyx

file_path = 's3://its-live-data.jpl.nasa.gov/icesat2/alt06/rel003/'
# file_path = '/Users/lipovsky/is2-data/' + shelf_name + '/'

import os.path
import h5py
import pickle

filelist_file_name = shelf_name + '_filelist.pickle'
if os.path.isfile(filelist_file_name):
    with open(filelist_file_name, 'rb') as handle:
        file_list = pickle.load(handle)
else:       
    import icepyx as ipx
    date_range = ['2018-10-14','2020-12-01']
    region_a = ipx.Query('ATL06', spatial_extent, date_range)
    granules=region_a.granules.avail

    file_list = []
    for f in granules: 
        file_list.append(file_path + f['producer_granule_id'])
        
    with open(filelist_file_name, 'wb') as handle:
        pickle.dump(file_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    print('Filelist written with %d files in it.'%len(file_list))

## Step 1. Read the ATL06 files into a Python data structure
Put the needed info in a dictionary, save the whole thing to file.

In [ ]:
import arc
import numpy as np
import importlib
importlib.reload(arc)

atl06_file_name = shelf_name + '_atl06.pickle'
# maskfile = '/Users/lipovsky/Downloads/BedMachineAntarctica_2020-07-15_v02.nc'
maskfile = 'BedMachineAntarctica_2020-07-15_v02.nc'

if os.path.isfile(atl06_file_name):
    print('Data already pickled, so theres no need to ingest data. \
To repeat the data ingest, it would probably be best to change the filename of the \
existing pickle file.')
else:
    arc.ingest(file_list,atl06_file_name,maskfile)

## Step 2.  Run the rift detector

In [ ]:
# Load data (deserialize)
with open(atl06_file_name, 'rb') as handle:
    atl06_data = pickle.load(handle)

# for i in range(len(atl06_data['quality'])):
#     atl06_data['quality'][i] = np.array(atl06_data['quality'][i])
#     atl06_data['h'][i] = np.array(atl06_data['h'][i])
#     atl06_data['geoid'][i] = np.array(atl06_data['geoid'][i])
#     atl06_data['azimuth'][i] = np.array(atl06_data['azimuth'][i])
#     atl06_data['h_sig'][i] = np.array(atl06_data['h_sig'][i])


# Find the rifts
rift_obs = arc.get_rifts(atl06_data)

# Store the rifts in a dataframe
import pandas as pd
import geopandas
rift_obs=pd.DataFrame(rift_obs)
rift_obs = geopandas.GeoDataFrame(rift_obs,
                             geometry=geopandas.points_from_xy(rift_obs['x-centroid'],
                                                               rift_obs['y-centroid']))

## Step 3. Save the rift_obs

In [ ]:
rift_obs_output_file_name = shelf_name + '_rift_obs.pickle'
with open(rift_obs_output_file_name, 'wb') as handle:
    pickle.dump(rift_obs, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Additional Steps.

From this point, useful next steps are to quality control the data (notebook qc.ipynb) and to analyze the data (analyze_rift_measurements.ipynb)